In [43]:
# Sanity check for D-Wave systems :)
import numpy as np
from dwave.optimization import Model
model = Model()
i = model.list(5)
c = model.constant([5, 4, 3, 1, 1])
y = (i * c).sum()
model.minimize(y)

from dwave.system import LeapHybridNLSampler
sampler = LeapHybridNLSampler()
results = sampler.sample(
    model,
    label='example')

route, = model.iter_decisions()
print(route.state(0))

SolverFailureError: Problem not accepted because user has insufficient remaining solver access time in project cSXm.

In [44]:
from dwave.system import LeapHybridNLSampler
from dwave.optimization.model import Model
import numpy as np
import resources

# Step 1: Define the flow rate and distance matrices
f_matrix = resources.flows  # The flow rate between rooms
d_matrix = resources.distances  # The distance between locations

def cost_function(n, solution, flow_matrix, distance_matrix):
    cost = 0
    for i in range(n):
        for j in range(n):
            cost += flow_matrix[i][j] * distance_matrix[int(solution[i])][int(solution[j])]
    return cost


# Step 2: Initialize the model
def optimize(flow_matrix, distance_matrix, iterations):
    
    optimal_route = None

    for i in range(iterations):
        model = Model()

        # Step 3: Define matrix constants: flow and distances
        distances = model.constant(distance_matrix)
        flows = model.constant(flow_matrix)

        # Step 4: Add decision variables (rooms assigned to locations)
        num_rooms = len(flow_matrix)
        route = model.list(num_rooms)  # The route represents the permutation of rooms

        # Step 5: Define the nonlinear objective
        # Minimize the cost, considering the flow rate and distance between rooms
        # Cost function: sum(cost * distances[route[i], route[j]] * flows[i, j])
        # where route[i] is the room placed at location i, and route[j] is the room placed at location j.
        ixs = model.constant(list(range(num_rooms)))
        jxs = model.constant(list(range(num_rooms)))
        alpha = model.constant(25)
        beta = model.constant(2)
        gamma = model.constant(200000)

        rooms_per_floor = [8, 6, 5]
        accumulated_rooms_per_floor = [0]
        for i in range(len(rooms_per_floor)):
            accumulated_rooms_per_floor.append(accumulated_rooms_per_floor[i] + rooms_per_floor[i])
        height_m = np.zeros((num_rooms, num_rooms))
        for i in range(num_rooms):
            for j in range(num_rooms):
                h_i = -1
                while accumulated_rooms_per_floor[h_i + 1] <= i:
                    h_i += 1
                h_j = -1
                while accumulated_rooms_per_floor[h_j + 1] <= j:
                    h_j += 1
                height_m[i][j] = abs(h_i - h_j)
        height = model.constant(height_m) 

        # Large initial value of cost

        cost = 9999999999
        model.minimize(
            ((distances[route[ixs], route[jxs]] * flows[ixs, jxs]).sum()).sum() + alpha * distances[route[ixs], route[jxs]].sum()
            + beta * flows[ixs, jxs].sum() * flows[ixs, jxs].sum() + gamma * height[ixs, jxs].sum()
        ) 

        # Step 6: Solve the model using D-Wave's sampler
        sampler = LeapHybridNLSampler()

        # Create the EmbeddingComposite to find an embedding onto the hardware
        results = sampler.sample(model, label="hi")

        results = sampler.sample(model, label="bye")

        route, = model.iter_decisions()
        temp_cost = cost_function(num_rooms, route.state(0), flow_matrix, distance_matrix)
        if temp_cost < cost:
            cost = temp_cost
            optimal_route = route.state(0)

    return optimal_route, cost

_, final_cost = optimize(f_matrix, d_matrix, 3)
print(f"cost = {final_cost}")
optimal_route = [13,8,10,9,11,4,2,7,3,12,1,0,6,15,17,16,18,5,14]
optimal_cost = cost_function(19, optimal_route, f_matrix, d_matrix)
print(f"optimal cost = {optimal_cost}")
percent_diff = float(final_cost - optimal_cost) / float(final_cost)
print(percent_diff)

cost = 63485135.275898494
optimal cost = 21985693.055648006
0.6536875449646453


In [19]:
# Time Evolution
total_time = 12 # 1 year
f_matrix = resources.flows
d_matrix = resources.distances
n = len(f_matrix)

iterations = 1 # Customizable based on choice

for t in range(total_time):
    # Find optimal solution for this month
    print(f"Optimal Solution in month {t}: ")
    print(optimize(f_matrix, d_matrix, iterations))

    # Vary flow rates over time by scaling each one by a random number sampled from normal distribution ranging from scaling factors of 0.9 to 1.1 (10%)
    variation = np.zeros((n, n))
    for j in range(n):
        for k in range(j+1, n):
            while variation[j][k] < 0.9 or variation[j][k] > 1.1:
                variation[j][k] = np.random.normal(1, .05)
            
            # Scale each flow by corresponding variation factor and correct it due to symmetric nature
            f_matrix[j][k] = f_matrix[j][k] * variation[j][k]
            f_matrix[k][j] = f_matrix[j][k]

Optimal Solution in month 0: 
(array([15., 18.,  2., 17.,  9.,  7., 10.,  6.,  5.,  8., 11., 14.,  0.,
        4., 16., 12.,  3.,  1., 13.]), 67020578)
Optimal Solution in month 1: 
(array([ 5., 13.,  1.,  7.,  2., 16.,  9.,  3., 10.,  8., 14., 11., 15.,
       17.,  4.,  6.,  0., 12., 18.]), np.float64(72821831.96271816))
Optimal Solution in month 2: 
(array([ 8., 14.,  3.,  4., 13.,  2., 15.,  9.,  7., 18., 16., 11., 17.,
        6.,  1., 10.,  0., 12.,  5.]), np.float64(58688039.95700801))
Optimal Solution in month 3: 
(array([ 0., 18., 10.,  6.,  3.,  2.,  5., 15., 12.,  8., 13.,  9.,  4.,
        1., 14., 17., 16.,  7., 11.]), np.float64(62571405.94062279))
Optimal Solution in month 4: 
(array([ 2., 10., 12., 17.,  1.,  7., 14., 15., 11.,  6., 16.,  0.,  4.,
        3., 18.,  8.,  9.,  5., 13.]), np.float64(44840438.00096479))
Optimal Solution in month 5: 
(array([ 1.,  2., 17., 15., 14.,  7., 10., 13.,  8.,  9.,  0.,  4., 11.,
       16.,  6., 12.,  5., 18.,  3.]), np.float64(506